In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import random as rnd

In [2]:
train = pd.read_csv("spotify_train.csv")
test = pd.read_csv("spotify_test.csv")

In [3]:
# Entropy and gini functions
def gini(x):
    return 1 - np.sum(np.square(x.value_counts()/len(x)))

def entropy(x):
    p = x.value_counts()/len(x)
    return -np.sum(np.log2(p) * p)

In [4]:
# Finds the optimal split for float values
def opt_split_float(data, split_field, analysis_field, criterion=gini):
    # Generates a list of the highest left or right purity values for each split
    # When selecting for lowest value, it will ensure a low average while ignoring scenarios where left split is very small
    # and right split is very large
    p_vals = {}
    for i in data[split_field]:
        left_data = data[data[split_field] <= i][analysis_field]
        right_data = data[data[split_field] > i][analysis_field]
        
        # Assigns the purity value based on the highest of left and right
        p_vals[i] = max(criterion(left_data), criterion(right_data))
        #p_vals[current_slice] = (left_p_val + right_p_val)/2
        
        
    return min(zip(p_vals.values(), p_vals.keys()))
        

In [39]:
# Determines the optimal split for boolean values
def opt_split_bool(data, split_field, analysis_field, criterion=gini):
    return ((criterion(data[data[split_field] <= 0][analysis_field]) + criterion(data[data[split_field] > 0][analysis_field]))/2, 0)

In [40]:
# Determines a generalized version of the optimal split
def opt_split(data, split_field, analysis_field, split_type, criterion=gini):
    if split_type == "bool":
        return opt_split_bool(data, split_field, analysis_field, criterion=criterion)
    else:
        return opt_split_float(data, split_field, analysis_field, criterion=criterion)

In [45]:
# Trains a model using the given data, field dictionary, and the given parameters
def train_tree(train, fields, criterion=gini, max_depth=None, min_instances=2, target_impurity=0.0, current_depth=0):
    # Allows the max depth to be None without blowing things up
    if max_depth is None:
        max_depth = 9999999
        
    # Stopping Condition
    if train is None or len(train) == 0: 
        return None
    
    # Gets the gini and the majority count for the current node
    gi = gini(train["track_genre"])
    mc = train["track_genre"].value_counts().index[0]
    
    # Additional stopping conditions
    if gi <= target_impurity or len(train) < min_instances or current_depth >= max_depth:
        return (None, None, len(train), mc, gi, current_depth, None, None)
    
    # Iterates through the list of fields, finds the optimal split for each, and finds the lowest gini value in total
    opt_ci = 99999999
    opt_field = ""
    opt_split_val = 0
    print("Iterating through fields, depth is ", current_depth)
    for field in fields.keys():
        field_ci = opt_split(train, field, "track_genre", fields[field], criterion=criterion)
        
        if (field_ci[0] < opt_ci):
            opt_ci = field_ci[0]
            opt_field = field
            opt_split_val = field_ci[1]
    
    # Creates the left and right splits
    left_split = train_tree(train[train[opt_field] <= opt_split_val], fields, max_depth=max_depth, min_instances=min_instances, target_impurity=target_impurity, current_depth=current_depth+1)
    right_split = train_tree(train[train[opt_field] > opt_split_val], fields, max_depth=max_depth, min_instances=min_instances, target_impurity=target_impurity, current_depth=current_depth+1)
    return (opt_field, opt_split_val, len(train), mc, gi, current_depth, left_split, right_split)

In [22]:
# Determines whether or not a single tuple is accurate
def predict_row(model, row):
    if model is None:
        return None
    elif model[0] is None:
        return model[3]
    else:
        if row[model[0]] <= model[1]:
            p = predict_row(model[6], row)
        else:
            p = predict_row(model[7], row)
            
        if p is None:
            return model[3]
        else:
            return p

# Determines the accuracy of the given dataset
def predict(model, data):
    predictions = []
    for i in range(len(data)):
        predictions.append(predict_row(model, data.iloc[i]))
        
    return predictions

In [15]:
def accurracy(true, pred):
    total = 0
    for i, j in zip(true, pred):
        if i == j:
            total += 1
    return total/len(true)

In [9]:
field_dict = {"duration_ms":"float", "energy":"float", "mode":"bool", "popularity":"float", "explicit":"bool", 
              "loudness":"float", "speechiness":"float", "acousticness":"float", "instrumentalness":"float", 
              "liveness": "float", "valence": "float", "tempo": "float"}

In [59]:
# Generates a list of errors for n fold with the given parameters
def validate(folds, max_depth, criterion, min_instances, target_impurity):
    acc_list = []
    
    # Validates 10 times
    for f in range(folds):
        train_fold = train[train.index % folds != f]
        valid_fold = train[train.index % folds == f]
        model = train_tree(train_fold, field_dict, max_depth=max_depth, min_instances=min_instances, criterion=criterion, target_impurity=target_impurity)
        predictions = predict(model, valid_fold)
        acc_list.append(accurracy(valid_fold["track_genre"], predictions))
    
    # Returns the validation error
    return acc_list

In [60]:
acc_list = validate(10, 8, entropy, 2, 0)

Iterating through fields, depth is  0
Iterating through fields, depth is  1
Iterating through fields, depth is  2
Iterating through fields, depth is  3
Iterating through fields, depth is  4
Iterating through fields, depth is  5
Iterating through fields, depth is  6
Iterating through fields, depth is  7
Iterating through fields, depth is  7
Iterating through fields, depth is  6
Iterating through fields, depth is  7
Iterating through fields, depth is  7
Iterating through fields, depth is  5
Iterating through fields, depth is  6
Iterating through fields, depth is  7
Iterating through fields, depth is  4
Iterating through fields, depth is  5
Iterating through fields, depth is  6
Iterating through fields, depth is  7
Iterating through fields, depth is  3
Iterating through fields, depth is  4
Iterating through fields, depth is  5
Iterating through fields, depth is  6
Iterating through fields, depth is  7
Iterating through fields, depth is  5
Iterating through fields, depth is  6
Iterating th

Iterating through fields, depth is  3
Iterating through fields, depth is  4
Iterating through fields, depth is  5
Iterating through fields, depth is  6
Iterating through fields, depth is  7
Iterating through fields, depth is  7
Iterating through fields, depth is  6
Iterating through fields, depth is  7
Iterating through fields, depth is  7
Iterating through fields, depth is  5
Iterating through fields, depth is  6
Iterating through fields, depth is  7
Iterating through fields, depth is  4
Iterating through fields, depth is  5
Iterating through fields, depth is  6
Iterating through fields, depth is  7
Iterating through fields, depth is  7
Iterating through fields, depth is  3
Iterating through fields, depth is  4
Iterating through fields, depth is  5
Iterating through fields, depth is  6
Iterating through fields, depth is  5
Iterating through fields, depth is  6
Iterating through fields, depth is  7
Iterating through fields, depth is  6
Iterating through fields, depth is  7
Iterating th

Iterating through fields, depth is  6
Iterating through fields, depth is  7
Iterating through fields, depth is  7
Iterating through fields, depth is  5
Iterating through fields, depth is  6
Iterating through fields, depth is  6
Iterating through fields, depth is  7
Iterating through fields, depth is  4
Iterating through fields, depth is  5
Iterating through fields, depth is  6
Iterating through fields, depth is  7
Iterating through fields, depth is  3
Iterating through fields, depth is  4
Iterating through fields, depth is  5
Iterating through fields, depth is  6
Iterating through fields, depth is  6
Iterating through fields, depth is  7
Iterating through fields, depth is  4
Iterating through fields, depth is  5
Iterating through fields, depth is  2
Iterating through fields, depth is  3
Iterating through fields, depth is  4
Iterating through fields, depth is  5
Iterating through fields, depth is  6
Iterating through fields, depth is  7
Iterating through fields, depth is  3
Iterating th

Iterating through fields, depth is  7
Iterating through fields, depth is  7
Iterating through fields, depth is  5
Iterating through fields, depth is  4
Iterating through fields, depth is  5
Iterating through fields, depth is  6
Iterating through fields, depth is  6
Iterating through fields, depth is  7
Iterating through fields, depth is  7
Iterating through fields, depth is  5
Iterating through fields, depth is  6
Iterating through fields, depth is  7
Iterating through fields, depth is  6
Iterating through fields, depth is  7
Iterating through fields, depth is  1
Iterating through fields, depth is  2
Iterating through fields, depth is  3
Iterating through fields, depth is  4
Iterating through fields, depth is  4
Iterating through fields, depth is  5
Iterating through fields, depth is  6
Iterating through fields, depth is  7


In [67]:
print(acc_list)

[0.71, 0.665, 0.7, 0.675, 0.725, 0.72, 0.725, 0.71, 0.665, 0.73]


In [62]:
print("Validation Error = +- ", (max(acc_list) - min(acc_list))/2)

Validation Error = +-  0.03249999999999997


In [63]:
# Trains the final model
model = train_tree(train, field_dict, max_depth=8, criterion=entropy, min_instances=2, target_impurity=0)

Iterating through fields, depth is  0
Iterating through fields, depth is  1
Iterating through fields, depth is  2
Iterating through fields, depth is  3
Iterating through fields, depth is  4
Iterating through fields, depth is  5
Iterating through fields, depth is  6
Iterating through fields, depth is  7
Iterating through fields, depth is  7
Iterating through fields, depth is  6
Iterating through fields, depth is  7
Iterating through fields, depth is  7
Iterating through fields, depth is  5
Iterating through fields, depth is  6
Iterating through fields, depth is  7
Iterating through fields, depth is  4
Iterating through fields, depth is  5
Iterating through fields, depth is  6
Iterating through fields, depth is  7
Iterating through fields, depth is  3
Iterating through fields, depth is  4
Iterating through fields, depth is  5
Iterating through fields, depth is  6
Iterating through fields, depth is  6
Iterating through fields, depth is  7
Iterating through fields, depth is  7
Iterating th

In [64]:
# Predicts the values of test
predictions = predict(model, test)

In [65]:
# Finds the accuracy of test
accurracy(test["track_genre"], predictions)

0.692

Format: (feature name, feature value threshold, number of elements below, majority class, impurity score, depth, left, right)

('explicit', 0, 2000, 'rock', 0.6666415, 0, 
    ('popularity', 59.0000000000003, 1727, 'rock', 0.6597971050742507, 1, 
        ('speechiness', 0.08058880000000003, 960, 'rock', 0.6132530381944444, 2, 
            ('popularity', 54.044000000000224, 744, 'rock', 0.5318100358422939, 3, 
                ...),
            ('instrumentalness', 0.012419999999999997, 216, 'hip-hop', 0.5259344993141291, 3, 
                ...),
        ('popularity', 64.07999999999991, 767, 'pop', 0.6114137779220756, 2, 
            ('duration_ms', 348353.7919999959, 150, 'hip-hop', 0.3121777777777778, 3,
                ...),
            ('acousticness', 0.005908238199999999, 617, 'pop', 0.555445521147183, 3,
                ...),
    ('energy', 0.9656399999999898, 273, 'hip-hop', 0.4280481423338567, 1, 
        ('instrumentalness', 0.09913800000000011, 268, 'hip-hop', 0.4096402316774338, 2, 
            ('speechiness', 0.0283328, 263, 'hip-hop', 0.3895097514782634, 3,
                ...),
            (None, None, 5, 'rock', 0.0, 3, None, None)), 
        (None, None, 5, 'rock', 0.0, 2, None, None)))